In [ ]:
#first of all we have to import data to work on
#so the thing about api that we learned before , now if the data set is really big it will take a lot of time to upload manually
#now we can use the api to directly import it on here which will be way faster than manually automating
!pip install kaggle


In [ ]:
#now we have to set the path of kaggle json file we have imported
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

**importing twitter sentiment dataset**



In [ ]:
# now we use the api to fetch the dataset
!kaggle datasets list -s "sentiment140 dataset"

ref                                                                 title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
kazanova/sentiment140                                               Sentiment140 dataset with 1.6 million tweets         81MB  2017-09-13 22:43:19         176275       1879  0.88235295       
ibrahimserouis99/twitter-sentiment-analysis-and-word-embeddings     Twitter Sentiment Analysis and Word Embeddings      521MB  2022-08-02 20:47:34            620         24  0.9411765        
bhavikjikadara/tweets-dataset                                       Tweets Dataset                                       53MB  2024-02-09 17:40:21           1046         31  1.0              
milobele/sentiment140-dataset-1600000-tw

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#now the dataset we got is in zip file, which was pretty fast using this method now we extract the compressed file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip: #we are reading the datafile with zip
  zip.extractall()
  print('extraction is completed bro')



extraction is completed bro


In [ ]:
#now we need libraries that would help us in doing sentiment analysis
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords #natural language toolkit  , corpus(means storage of data) the the model
from nltk.stem.porter import PorterStemmer #we used this before in movie recommendation project, stemming is to reduce the word to base form
from sklearn.feature_extraction.text import TfidfVectorizer # making the machine to understand the data, converting text to numeric
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  #logistic
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords in english
print(stopwords.words('english')) #these text we remove that doesnt add importance or doesnt have textual importance
#we will remove these stop words from the data we import


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Now doing the data processing

In [ ]:
#loading the data from csv file to pandas dataframe
x_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1') #specifying the path and the encoding method


In [ ]:
#now vizualizing the data and getting info
x_data.shape

(1599999, 6)

In [ ]:
x_data.head()
#first value is read as column, and not the actual columbns in the file

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming the columns and reading data again
column_names = ['target','ids','date','flag','user','text']
x_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=column_names)
#

In [ ]:
x_data.shape


(1600000, 6)

In [ ]:
x_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting the number of missing values in the dataset
x_data.isnull().sum() #checking if there are missing values



,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
#the data is already clean so we no need to do processing
#now we have various distribution like 0 to 4
#now we check the distribution of target volumn
x_data['target'].value_counts()
#if the the data was not even then would do upsampling or downsampling right now we dont need to

,count
target,
0,800000
4,800000


convert the target 4 to 1

In [ ]:
x_data.replace({'target':{4:1}}, inplace = True)


In [ ]:
x_data['target'].value_counts()



,count
target,
0,800000
1,800000


0 - negative tweet
1 - positive tweet

doing the stemming procedure

stemming vs lemmatization ?

In [ ]:
port_stem = PorterStemmer()


In [ ]:
#now we create a function
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #we are replacing anything that is not a-z example hello world! to hello world only
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split() #splitting all the words and adding them to list later
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #performing stemming on each word iterating with loop
  stemmed_content = ' '.join(stemmed_content) #after converting we join everything in one sentence

  return stemmed_content

In [ ]:
x_data['stemmed_content'] = x_data['text'].apply(stemming) #50 minutes to complete this execution

In [ ]:
x_data.head() # now we have the column that is stemmed content, now we dont need others so we will remove that

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(x_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(x_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#separting the data and label
x = x_data['stemmed_content'].values
y = x_data['target'].values


In [ ]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(y)

[0 0 0 ... 1 1 1]


splitting the data to training data and test data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2) #we use stratify to basically make the proportions same for train and test data
#random state defines the number of ways the data is being devided


In [ ]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
#converting the textual data to numerical data now using vectorization

vectorizer = TfidfVectorizer()  #storing in a variable to call it later
#this function gives a certain importance based on how much the word is being repeated
x_train = vectorizer.fit_transform(x_train) #here in train we use fit to actually get the context of each word and transform into numerical value
x_test = vectorizer.transform(x_test) #this is test data so we dont use fit for this


In [ ]:
print(x_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

In [ ]:
print(x_test)

  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538
  (319995, 107868)	0.33399349737546963
  (319995, 109379)	0.3020896484890833
  (319995, 155493)	0.2770682832971669
  (319995, 2133

now we will train the machine learning model
**using logistic regression which is used for yes or no type of classification , in this case we have to classes which are positive and negative**

In [ ]:
model = LogisticRegression(max_iter=1000) # this sets maximum number of times the model can go through data

In [ ]:
model.fit(x_train,y_train) #so now we are feeding the data to model from which the model will be trained from

LogisticRegression(max_iter=1000)

now we are trying to get the model accuracy

In [ ]:
#basically to evaluate the model
#accuracy score on the training data
x_train_prediction = model.predict(x_train) #predict the training data labels
training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [ ]:
print('accuracy score of training data : ',training_data_accuracy)

accuracy score of training data :  0.79871953125


In [ ]:
#now we will check the accuracy on test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)


In [ ]:
print('accuracy score for testing data: ', test_data_accuracy)

accuracy score for testing data:  0.77668125


saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))


using the saved model for future prediction that we have created

In [ ]:
#loading the model first
loaded_model = pickle.load(open('trained_model.sav','rb'))


In [ ]:
x_new = x_test[3]
print(y_test[3])

prediction = loaded_model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('the tweet is negative')
else:
  print('the tweet is positive')

0
[0]
the tweet is negative


In [ ]:
# Load the trained model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

# Define the stemming function again
def preprocess_input(text):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', text)
    stemmed_content = stemmed_content.lower().split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    return ' '.join(stemmed_content)

# Vectorize and predict on new input
def predict_sentiment(text):
    processed_text = preprocess_input(text)
    vectorized_text = vectorizer.transform([processed_text])  # Vectorize the preprocessed input text
    prediction = loaded_model.predict(vectorized_text)

    if prediction[0] == 0:
        return 'The sentiment is negative.'
    else:
        return 'The sentiment is positive.'

# Take input from the user
user_input = input("Enter a sentence to analyze sentiment: ")
print(predict_sentiment(user_input))


Enter a sentence to analyze sentiment: i am a really good guy 
The sentiment is positive.
